**<font color = black size=6>实验五:决策树</font>**

**<font color = blue size=4>第一部分:函数介绍</font>**

In [53]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

1)Counter类的使用

In [54]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
#使用Counter类对数组第2列进行遍历
counter = Counter(x[:,1])
#第2列中有1个132和2个133，输出该counter对象可以统计这列的数值情况，便于之后的统计
print(counter)
#因为第2列中没有为0的值，所以返回0
print(counter[0])
#因为第2列中有2个133，所以返回2
print(counter[133])
#一般的字典操作方法都能在该类中使用，例如可以通过values函数返回该列的非重复值的个数，方便对某列的非重复值的个数进行查看
print(counter.values())
#可以输出所有非重复值
print(list(counter))

Counter({133: 2, 132: 1})
0
2
dict_values([2, 1])
[133, 132]


2)使用numpy中的unique实现非重复值的提取

In [55]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
a=np.unique(x[:])
a1=np.unique(x[:,1])
print(a,a1)

[  0   1 132 133] [132 133]


**<font color = blue size=4>第二部分:实验任务</font>**

**<font color = blue size=3>1) 任务一</font>**

任务一要求完成不同量化标准(信息增益, 信息增益率, 基尼系数)下的结点划分函数

<span style="color:purple">该数据集(train_titanic.csv)为分类数据集，为泰坦尼克号的部分乘客信息以及最后是否生还。包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

In [56]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

<span style="color:purple">1) 使用pandas库将训练数据集'train_titanic.csv'载入到Dataframe对象中</span>

In [57]:
# load dataset
df_train = pd.read_csv('./train_titanic.csv')
print(df_train[:3])



   Sex  sibsp  Parch  Pclass  Survived
0    0      1      0       3         0
1    1      1      0       1         1
2    1      0      0       3         1


<span style="color:purple">2) 编写函数，给定任何标签数组计算其信息熵  
    输入：标签数组  
    输出：该数组对应的信息熵  
    计算信息熵公式:
某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$</span>
    
    
<span style="color:purple">例:  
    输入:[[0],[1]]   
    输出:(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)+(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)=1</span>

In [58]:
def entropy(label):
    ent = 0
    # 统计
    c = Counter(label)
    num = c.values()
    p = np.array(list(num))/sum(num)
    # print(p)
    # calculate entropy
    ent = -(p*np.log2(p)).sum()
    return ent

ent = entropy(df_train['Sex'])
print('ent',ent)

# ent = entropy([1.,1.,1.])
# print('ent',ent)

ent 0.9481042889614961


<span style="color:purple">3) 编写函数，函数功能为将所给的数据集按照指定维度的特征进行划分为若干个不同的数据集  
    【输入】：特征集合，标签集合，指定维度  
    【输出】：划分后所得到的子树属性集合，子树标记集合</span>

<span style="color:purple">例子:  
    【输入】:特征集合:[[0,0,0],[0,0,1],[1,0,2]]  
    标签集合:[[0],[1],[2]]  
    指定维度:0</span>  
    
<span style="color:purple">【输出】:[[0,0,0],[0,0,1]]和[[1,0,2]]  
    [[0],[1]]和[[2]]  
    tips:即将特征按其第0维度进行划分，由于第0维特征有0和1两个不同的数值，所以特征集合划分为[[0,0,0],[0,0,1]]和[[1,0,2]]，标签集合划分为[[0],[1]]和[[2]]</span>

In [86]:
def split(feature, label, d):
    # print(feature)
    # print(label)
    split_feature = []
    split_label = []
    
    # get feature of the dim
    feature_specified_dim = feature[feature.columns[d]]
    # print(feature_specified_dim)
    c = Counter(feature_specified_dim)
    # print(c)

    for key in c.keys():
        key = int(key)
        _index = [i for (i, v) in enumerate(feature_specified_dim)
                 if key==v]
        split_feature.append(feature.iloc[_index])
        split_label.append(label.iloc[_index])
    
    # print('split_feature\n',split_feature)
    # print('split_label\n',split_label)
    return split_feature,split_label

label = df_train['Survived'][:5]
feature = df_train.drop(['Survived'],axis=1)[:5]
sp = split(feature,label,0)
print(sp)

([   Sex  sibsp  Parch  Pclass
0    0      1      0       3
4    0      0      0       3,    Sex  sibsp  Parch  Pclass
1    1      1      0       1
2    1      0      0       3
3    1      1      0       1], [0    0
4    0
Name: Survived, dtype: int64, 1    1
2    1
3    1
Name: Survived, dtype: int64])


<span style="color:purple">4) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该特征集合中信息增益(使用ID3算法中的公式计算)【最大】的特征  
    输入：特征集合，标签集合  
    输出：该次划分的最佳信息增益值，最佳划分维度  
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，其中值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征值对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$
该函数中你需要计算出每个特征的信息增益并输出，然后返回最佳的信息增益值和对应的特征的维数</span>

In [60]:
def one_split_ID3(x_data, y_label):
    # init
    best_entropy = 0
    best_dimension = -1
    # calculate shape
    num_samples, num_features = x_data.shape
    # calculate ent
    without_ent = entropy(df_train['Survived'])
    # iter for each feature
    for dim in range(num_features):
        split_feature,split_label = split(feature,label,dim)
        with_ent = 0
        # each group
        for group_label in split_label:
            
            _ent = entropy(group_label)
            # print('_ent',_ent)
            
            percent = len(group_label)/len(y_label)
            # print('percent',percent)
            
            with_ent += percent*_ent
        
        # calculate the max entropy add
        if(without_ent-with_ent>best_entropy):
            best_entropy = without_ent-with_ent
            best_dimension = dim
                 
                    
    return best_entropy, best_dimension

label = df_train['Survived'][:10]
feature = df_train.drop(['Survived'],axis=1)[:10]
best_entropy, best_dim = one_split_ID3(feature,label)
print('best_entropy, best_dim',best_entropy, best_dim)

best_entropy, best_dim 0.7709131174627031 0


<span style="color:purple">5) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该特征集合中信息增益率(使用C4.5算法中的公式计算)【最大】的特征  
    输入：特征集合，标签集合  
    输出：最佳划分的信息增益率值，对应的划分维度  
    计算信息增益率公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征类型feature,该特征包含V个不同的取值，值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^V|D^v|=|D|)$,该特征本身的信息熵为Ent(feature),则该特征值对应的信息增益率为$$Gain\_ratio(D,feature)=\frac{Gain(D,feature)}{Ent(feature)}$$
该函数中你需要输出每个特征的信息增益率，之后返回最佳的信息增益率和对应的特征维数</span>

In [61]:
def one_split_C4_5(x_data, y_label):
    # init
    best_entropy = 0
    best_dimension = -1
    # calculate shape
    num_samples, num_features = x_data.shape
    # calculate ent
    without_ent = entropy(df_train['Survived'])
    # iter for each feature
    for dim in range(num_features):
        split_feature,split_label = split(feature,label,dim)
        with_ent = 0
        feature_entropy = 0
        # each group
        for group_label in split_label:
            
            _ent = entropy(group_label)
            # print('_ent',_ent)
            
            percent = len(group_label)/len(y_label)
            # print('percent',percent)
            
            with_ent += percent*_ent
        
        # calculate feature entropy
        for group_label in split_feature:
            ratio = len(group_label)/len(y_label)
            feature_entropy += -ratio*np.log2(ratio)
        
        # print('feature_entropy',feature_entropy)
        
        # c4.5
        with_ent_ratio = (without_ent-with_ent)/feature_entropy
        # print('with_ent_ratio',with_ent_ratio)
        
        # calculate the max entropy add
        if(with_ent_ratio>best_entropy):
            best_entropy = with_ent_ratio
            best_dimension = dim
                 
                    
    return best_entropy, best_dimension


label = df_train['Survived'][:10]
feature = df_train.drop(['Survived'],axis=1)[:10]
best_entropy, best_dim = one_split_C4_5(feature,label)
print('best_entropy, best_dim',best_entropy, best_dim)

best_entropy, best_dim 0.7709131174627031 0


<span style="color:purple">6) 编写函数，进行【一次】决策树的结点划分，遍历找出该特征集合中基尼系数(使用CART算法中的公式计算)最小的特征以及最佳的划分值  
    输入：特征集合，标签集合  
    输出：最佳的基尼系数，对应的划分维度，最佳划分值  
    计算基尼系数公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,该集合中标签类别总共有K类，第k类样本所占比例为$p_k$(k=1,2,..,K),则该数据集对应的基尼系数为$$Gini(D)=1-\sum_{k=1}^K{p_k}^2$$  
    而取其中一个特征feature，选定该特征的一个值value，根据该特征的值是否为value将数据集分为两个子集$D_1$和$D_2$,则该特征对应的基尼系数为$$Gini\_index(D,feature)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$$
该函数中你需要遍历每一列特征，找出每列中的非重复值，计算出每个非重复值的基尼系数，返回最小的基尼系数、对应的特征维数和非重复值（分类值）。</span>

In [62]:
import itertools

def calculate_gini(y_label):
    c = Counter(y_label)
    gini = 1
    # print('c',c)
    for k,v in c.items():
        # print('k,v',k,v)
        gini -= (v/len(y_label))**2
    
    return gini

def one_split_CART(x_data, y_label):
    # init
    best_gini = 1
    best_dimension = -1
    best_combination_0 = []# include and higher is a split
    # calculate shape
    num_samples, num_features = x_data.shape
    # iter for each feature
    for dim in range(num_features):
        gini_index = 0
        _feature = x_data[x_data.columns[dim]]
        # print(_feature)
        c = Counter(_feature)
        # print('c',c)
        
        combinations = []
        for r in range(1, len(c.keys())+1):
            combinations.extend(list(itertools.combinations(c.keys(), r)))

        for combination in combinations:
            
            class_0_index = [i for (i,v) in enumerate(_feature) if v in combination]
            class_1_index = _feature.drop(class_0_index).index   
            # print('class_0_index',class_0_index)  
            # print('class_1_index',class_1_index)  
            
            # calculate gini
            if(len(class_0_index)!=0 and len(class_1_index)!=0):
                label_class_0 = y_label[class_0_index]
                label_class_1 = y_label[class_1_index]
                # print('label_class_0',label_class_0)
                # print('label_class_1',label_class_1)
                gini_class_0 = calculate_gini(label_class_0)
                gini_class_1 = calculate_gini(label_class_1)
                gini_index = len(class_0_index)/num_samples*gini_class_0\
                    + len(class_1_index)/num_samples*gini_class_1
                    
                # compare
                if(gini_index<best_gini):
                    best_gini = gini_index
                    best_dimension = dim
                    best_combination_0 = combination
                 
    return best_gini, best_dimension, best_combination_0

label = df_train['Survived']
feature = df_train.drop(['Survived'],axis=1)
best_gini, best_dim, best_combination_0 = one_split_CART(feature,label)
print('best_gini, best_dim, best_combination_0',best_gini, best_dim, best_combination_0)

best_gini, best_dim, best_combination_0 0.29649157246415725 0 (0,)


<span style="color:purple">7) 应用之前你在第4、5、6个部分编写的三个函数，在训练数据集'train_titanic.csv'上依次使用这些函数进行【一次】结点划分，并输出对应的最佳特征维数以及相应的信息增益值/信息增益率/(基尼系数和分类值)</span>

In [63]:
label = df_train['Survived'][:15]
feature = df_train.drop(['Survived'],axis=1)[:15]
best_entropy_C4_5, best_dim_C4_5 = one_split_C4_5(feature,label)
best_entropy_ID3, best_dim_ID3 = one_split_ID3(feature,label)
best_gini_CART, best_dim_CART, best_combination_0_CART = one_split_CART(feature,label)

print('best_entropy_C4_5, best_dim_C4_5',best_entropy_C4_5, best_dim_C4_5)
print('best_entropy_ID3, best_dim_ID3',best_entropy_ID3, best_dim_ID3)
print('best_gini_CART, best_dim_CART, best_combination_0_CART',best_gini_CART, best_dim_CART, best_combination_0_CART)

# 进行一次划分 以ID3为例
split_feature_ID3,split_label_ID3 = split(feature,label,best_dim_ID3)
print('split_feature_ID3',split_feature_ID3)
print('split_label_ID3',split_label_ID3)



best_entropy_C4_5, best_dim_C4_5 0.4825603121620572 0
best_entropy_ID3, best_dim_ID3 0.481012081089585 0
best_gini_CART, best_dim_CART, best_combination_0_CART 0.11666666666666667 0 (0,)
split_feature_ID3 [    Sex  sibsp  Parch  Pclass
0     0      1      0       3
4     0      0      0       3
5     0      0      0       3
6     0      0      0       1
7     0      3      1       3
12    0      0      0       3
13    0      1      5       3,     Sex  sibsp  Parch  Pclass
1     1      1      0       1
2     1      0      0       3
3     1      1      0       1
8     1      0      2       3
9     1      1      0       2
10    1      1      1       3
11    1      0      0       1
14    1      0      0       3]
split_label_ID3 [0     0
4     0
5     0
6     0
7     0
12    0
13    0
Name: Survived, dtype: int64, 1     1
2     1
3     1
8     1
9     1
10    1
11    1
14    0
Name: Survived, dtype: int64]


**<font color = blue size=3>2) 任务二</font>**

任务二承接任务一，用【ID3】算法实现一棵完整的决策树。

<span style="color:purple">1) 整理任务一的代码，根据实验二要求做适当修改来编写函数，【从剩余的特征集A中】寻找使得信息增益最大的特征   
    输入：数据集D、剩余的特征集A    
    输出：最佳划分的特征维数    
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$</span>

In [64]:
def best_split(D:pd.DataFrame, A):
    
    label = D['Survived']
    feature = D.drop(['Survived'],axis=1)
    
    # init
    best_entropy = 0
    best_dimension = -1
    # calculate ent
    without_ent = entropy(label)
    # iter for each feature
    for dim in A:
        split_feature,split_label = split(feature,label,dim)
        with_ent = 0
        # each group
        for group_label in split_label:
            
            _ent = entropy(group_label)
            # print('_ent',_ent)
            
            percent = len(group_label)/len(label)
            # print('percent',percent)
            
            with_ent += percent*_ent
        
        # calculate the max entropy add
        if(without_ent-with_ent>best_entropy):
            best_entropy = without_ent-with_ent
            best_dimension = dim

    return best_dimension

<span style="color:purple">2) 完成DTree类中的TreeGenerate、train函数以完成决策树的构建。并完成DTree类中的predict函数来用构建好的决策树来对测试数据集进行预测并输出预测准确率。</span>

In [65]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, feature_index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.feature_index = feature_index # feature_index表示该分支结点的划分特征的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分特征的值为val的孩子结点

In [83]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = {} # 用于存储每个特征可能的取值
    
        
 
    '''
    TreeGenerate函数用于递归构建决策树，伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
     
    '''    
    
    def TreeGenerate(self, D, A:set):
        
        # 生成结点 node
        node = Node()

        
        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if
        print('A',A)
        print('possible_value',self.possible_value)
        label = D['Survived']
        # print('label',label)
        # belong to 1 class
        if(len(np.unique(label))==1):
            node.isLeaf = True
            leaf_label = np.unique(label)[0]
            print('leaf_label',leaf_label)
            node.label = leaf_label
            return node
        
        
        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        A_null=False
        A_same = False
        if(len(A)==0):
            A_null=True
        else:
            count = 0
            for feature_index in A:
                feature = D[D.columns[feature_index]]
                # print('feature',feature)   
                c = Counter(feature)   
                # print('len(c.keys())',len(c.keys()))      
                if(len(c.keys())==1):
                    count+=1
            if(count==len(A)):
                A_same=True
        # judge
        if(A_same or A_null):
            node.isLeaf=True 
            c = Counter(label)
            leaf_label = c.most_common(1)[0][0]
            print('leaf_label',leaf_label)
            node.label = leaf_label
            return node
        
        
        # 从A中选择最优划分特征a_star
        # （选择信息增益最大的特征，用到上面实现的best_split函数） 
        best_dim = best_split(D,A)
        print('best_dim',best_dim)
        
        
        
        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for
        a_star = self.possible_value[best_dim]
        print('a_star',a_star)    
        for a_star_v in a_star:
            child = Node()
            name = D.columns[best_dim]
            print('name',name)
            _D = D[D[name]==a_star_v]
            print('_D',_D)
            if(len(_D)==0):
                child.isLeaf=True
                c = Counter(label)
                leaf_label = c.most_common(1)[0][0]
                child.label=leaf_label
            else:
                print('origin_A',A)
                new_A = A.copy()
                new_A.remove(best_dim)
                print('new_A',new_A)
                child = self.TreeGenerate(_D, new_A)
                
            node.isLeaf=False
            node.feature_index=best_dim
            node.addNode(a_star_v,child)
            
        return node
    
    
 
    '''
    train函数可以做一些数据预处理（比如Dataframe到numpy矩阵的转换，提取属性集等），并调用TreeGenerate函数来递归地生成决策树
 
    '''
    
    def train(self, D:pd.DataFrame):
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        A = set(range(D.shape[1] - 1)) # 特征集A
        
        #记下每个特征可能的取值
        for every in A:
            self.possible_value[every] = np.unique(np.array(D.iloc[:, every]))
        
        self.tree_root = self.TreeGenerate(D, A) # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        return

    
    '''
    predict函数对测试集D进行预测， 并输出预测准确率（预测正确的个数 / 总数据数量）
    
    '''
    
    def predict(self, D:pd.DataFrame):
        
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）      
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，

#         #则搜索x的划分特征为该行数据相应的特征值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的标签就是数据d的预测标签
        node = self.tree_root 
        num = D.shape[0]
        right_num = 0
        # iter
        for i in range(num):
            data_line = D.iloc[i,:]
            print('data_line',data_line)
            # visit tree
            assert isinstance(data_line, pd.Series)
            while(not node.isLeaf):
                dim = node.feature_index
                value = data_line[dim]
                print('value',value)
        
                node = node.children[value]
            # to leaf
            predict_label = node.label
            print('predict label',predict_label)
            label = data_line['Survived']
            print('true label',label)
            if(label==predict_label):
               right_num += 1 

        acc = right_num/num
        return acc


In [84]:
# 构建决策树
tree = DTree()
tree.train(df_train)

# 利用构建好的决策树对测试数据集进行预测，输出预测准确率（预测正确的个数 / 总数据数量）
test_df = pd.read_csv('./test_titanic.csv')
acc = tree.predict(test_df)
print('acc_rate',acc)


A {0, 1, 2, 3}
possible_value {0: array([0, 1], dtype=int64), 1: array([0, 1, 2, 3, 4, 5, 8], dtype=int64), 2: array([0, 1, 2, 3, 4, 5, 6, 9], dtype=int64), 3: array([1, 2, 3], dtype=int64)}
best_dim 0
a_star [0 1]
name Sex
_D       Sex  sibsp  Parch  Pclass  Survived
0       0      1      0       3         0
4       0      0      0       3         0
5       0      0      0       3         0
6       0      0      0       1         0
7       0      3      1       3         0
...   ...    ...    ...     ...       ...
998     0      1      1       1         0
1004    0      0      0       3         0
1006    0      0      0       3         0
1007    0      0      0       3         0
1008    0      1      1       3         0

[639 rows x 5 columns]
origin_A {0, 1, 2, 3}
new_A {1, 2, 3}
A {1, 2, 3}
possible_value {0: array([0, 1], dtype=int64), 1: array([0, 1, 2, 3, 4, 5, 8], dtype=int64), 2: array([0, 1, 2, 3, 4, 5, 6, 9], dtype=int64), 3: array([1, 2, 3], dtype=int64)}
best_dim 2
a_star [

In [74]:
#展示生成的决策树结构
def display_tree(node, indent=''):
    if node.isLeaf:
        print(indent + "Leaf Node: label =", node.label)
    else:
        print(indent + "Branch Node: feature_index =", node.feature_index)
        for value, child in node.children.items():
            print(indent + "|--> Child Value:", value)
            display_tree(child, indent + "   ")


display_tree(tree.tree_root)



Branch Node: feature_index = 0
|--> Child Value: 0
   Branch Node: feature_index = 2
   |--> Child Value: 0
      Branch Node: feature_index = 3
      |--> Child Value: 1
         Branch Node: feature_index = 1
         |--> Child Value: 0
            Leaf Node: label = 0
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
         |--> Child Value: 4
            Leaf Node: label = 0
         |--> Child Value: 5
            Leaf Node: label = 0
         |--> Child Value: 8
            Leaf Node: label = 0
      |--> Child Value: 2
         Branch Node: feature_index = 1
         |--> Child Value: 0
            Leaf Node: label = 0
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
         |--> Child Value: 4
            

**<font color = blue size=4>第三部分:作业提交</font>**

<span style="color:purple">本次实验分两周完成,实验报告提交地址(学号+姓名+实验五):https://send2me.cn/s0Yr-w4z/QCKd4ibz34miqQ   截止日期:10.20 14:20</span>

<span style="color:purple">实验课件获取地址:https://www.jianguoyun.com/p/DZRe9cwQp5WhChjz4p8FIAA</span>